In [16]:
import pandas as pd
import random
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.request import urlopen
import seaborn as sns
import scipy.optimize as sco

In [17]:
url=urlopen("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
soup = BeautifulSoup(url.read(), 'lxml')
tbody = soup.tbody
tr = tbody.find_all('tr')

In [18]:
data = []
for t in tr:
    data.append(t.text.split('\n'))

raw_df = pd.DataFrame(data)
raw_df.columns = raw_df.iloc[0,:]
raw_df = raw_df.iloc[1:,:]
listed_companies=raw_df['Symbol'].tolist()

In [19]:
data=pd.DataFrame()
for ticker in listed_companies:
    
    try:
        
        yahoo_data=yf.Ticker(ticker)
        stock_price=yahoo_data.history(start='2022-03-20',end='2023-03-07',interval='1d').reset_index()
        stock_price['Date']=stock_price['Date'].dt.tz_localize(None)
        stock_price=stock_price.set_index('Date')
        stock_price[ticker]=stock_price['Close']+stock_price['Dividends'].shift(periods=-1)
        data=pd.concat([data,stock_price[ticker]],axis=1)
        
    except Exception as e:
        pass
    

Got error from yahoo api for ticker BRK.B, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- BRK.B: No timezone found, symbol may be delisted
BF.B: No data found for this date range, symbol may be delisted


In [ ]:
esg_data=pd.DataFrame()

for ticker in listed_companies:
    ticker_name = yf.Ticker(ticker)
    try:
        if ticker_name.sustainability is not None: #if no response from Yahoo received, it will pass to next ticker
            ticker_df = ticker_name.sustainability.T #response dataframe
            ticker_df['symbol'] = ticker #adding new column 'symbol' in response df
            esg_data=pd.concat([esg_data,ticker_df],ignore_index=True)
    except (IndexError, ValueError) as e: #in case yfinance API misbehaves
        print(f'{ticker} did not run') #FYI
        pass

In [ ]:
new_esg_data=esg_data[['symbol','peerGroup','totalEsg']]
new_esg_data.columns=['Stock','Sector','ESG score']
new_esg_data.to_excel(r'C:\Users\niroo\Desktop\Jupyter\Projets\ESG.xlsx', index=False)

In [21]:
data.to_excel(r'C:\Users\niroo\Desktop\Jupyter\Projets\S&P5002.xlsx')